In [ ]:
# Global data variables
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/data/" 

In [ ]:
from pyspark.sql import functions as F



# Creación o modificación de columnas

En Spark hay un único método para la creación o modificación de columnas y es `withColumn`. Este método es de nuevo una transformación y toma dos parámetros: el nombre de la columna a crear (o sobreescribir) y la operación que crea la nueva columna.

Para una ejecución más óptima se recomienda utilizar únicamente las funciones de PySpark cuando se define la operación, pero como se detallará más adelante se pueden utilizar funciones propias. 

In [ ]:
movies_df = spark.read.csv(DATA_PATH + 'movie-ratings/movies.csv', sep=',', header=True, inferSchema=True)
ratings_df = spark.read.csv(DATA_PATH + 'movie-ratings/ratings.csv', sep=',', header=True, inferSchema=True)

In [ ]:
ratings_movies_df = ratings_df.join(movies_df, on='movieId', how='inner')

In [ ]:
ratings_movies_df.cache()



## Funciones de Spark



__valor fijo__

El ejemplo más sencillo es crear una columna con un valor fijo, en este caso, columna `now` con valor '2019/01/21 14:08', y columna `rating2`con valor 4.0.

Hint: `withColumn`

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('now', F.lit('2019/01/21 14:08'))

In [ ]:
ratings_movies_df.show(3)

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('rating2', F.lit(4.0))

In [ ]:
ratings_movies_df.show(3)



__duplicar columna__

In [ ]:
ratings_movies_df.withColumn('title2', F.col('title'))\
                 .select('title', 'title2')\
                 .show(10)



__operaciones aritmeticas__

In [ ]:
ratings_movies_df.withColumn('rating_10', F.col('rating') * 2)\
                 .select('rating', 'rating_10')\
                 .show(10)

In [ ]:
ratings_movies_df.withColumn('rating_avg', (F.col('rating') + F.col('rating2')) /  2)\
                 .select('rating', 'rating2', 'rating_avg')\
                 .show(10)

 

__if/else__

Crea la columna `kind_rating`, que sea 'high' en caso de que rating sea mayor que 4, y 'low' en caso contrario.

In [ ]:
ratings_movies_df.withColumn('kind_rating', 
                              F.when(F.col('rating') >= 4, 'high').otherwise('low')).show(10)



Se pueden concatenar multiples sentencias _when_. Esta vez, sobreescribe la columna `kind_rating` para crear un nivel intermedio, donde si es mayor que dos y menor que 4, `kind_rating` sea 'med'.

In [ ]:
ratings_movies_df.withColumn('kind_rating', 
                              F.when(F.col('rating') >= 4, 'high')\
                               .when(F.col('rating') >= 2, 'med')\
                               .otherwise('low')).show(20)



__operaciones con strings__

Pon en mayúsculas todos los títulos de las películas

In [ ]:
ratings_movies_df.withColumn('title', F.upper(F.col('title'))).show(3)



Extrae los 10 primeros caracteres de la columna `title`

In [ ]:
ratings_movies_df.withColumn('short_title', F.substring(F.col('title'), 0, 10))\
                 .select('title', 'short_title')\
                 .show(10)



Separa los diferentes géneros de la columna `genres` para obtener una lista, usando el separador '|'

In [ ]:
ratings_movies_df.withColumn('genres', F.split(F.col('genres'), '\|')).show(4)



Crea una nueva columna `1st_genre` seleccionando el primer elemento de la lista del código anterior

In [ ]:
ratings_movies_df.withColumn('1st_genre', F.split(F.col('genres'), '\|')[0])\
                 .select('genres', '1st_genre')\
                 .show(10)



Reemplaza el caracter '|' por '-' en la columna `genres`

In [ ]:
ratings_movies_df.withColumn('genres', F.regexp_replace(F.col('genres'), '\|', '-'))\
                 .select('title', 'genres')\
                 .show(10, truncate=False)



_Con expresiones regulares_

https://regexr.com/

In [ ]:
ratings_movies_df.withColumn('title', F.regexp_replace(F.col('title'), ' \(\d{4}\)', '')).show(5, truncate=False)

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('year', 
                                                 F.regexp_extract(F.col('title'),  '\((\d{4})\)', 1))

ratings_movies_df.show(5)



## Casting

Con el método `withColumn` también es posible convertir el tipo de una columna con la función `cast`. Es importante saber que en caso de no poder convertirse (por ejemplo una letra a número) no saltará error y el resultado será un valor nulo.

In [ ]:
ratings_movies_df.printSchema()



Cambia el formato de `year` a entero, y `movieId` a string.

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('year', F.col('year').cast('int'))
ratings_movies_df.show(5)

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('movieId', F.col('movieId').cast('string'))

In [ ]:
ratings_movies_df.printSchema()

In [ ]:
ratings_movies_df.withColumn('error', F.col('title').cast('int')).show(5)



## UDF (User Defined Functions)

Cuando no es posible definir la operación con las funciones de spark se pueden crear funciones propias usando la UDFs. Primero se crea  una función de Python normal y posteriormente se crea la UDFs. Es necesario indicar el tipo de la columna de salida en la UDF.

In [ ]:
from pyspark.sql.types import StringType, IntegerType, DoubleType, DateType



_Aumenta el rating en un 15% para cada película más antigua que 2000 (el máximo siempre es 5)._

In [ ]:
def increase_rating(year, rating):
    
    if year < 2000:
        rating = min(rating * 1.15, 5.0)
    
    return rating

In [ ]:
increase_rating_udf = F.udf(increase_rating, DoubleType())

In [ ]:
ratings_movies_df.withColumn('rating_inc', 
                              increase_rating_udf(F.col('year'), F.col('rating')))\
                 .select('title', 'year', 'rating', 'rating_inc')\
                 .show(20)



Extrae el año de la película sin usar expresiones regulares.

In [ ]:
title = 'Trainspotting (1996)'

In [ ]:
title.replace(')', '').replace('(', '')

In [ ]:
year = title.replace(')', '').replace('(', '').split(' ')[-1]
year = int(year)
year

In [ ]:
def get_year(title): 
    
    year = title.replace(')', '').replace('(', '').split(' ')[-1]
    if year.isnumeric():
        year = int(year)
    else:
        year = -1
    
    return year

In [ ]:
get_year_udf = F.udf(get_year, IntegerType())

In [ ]:
ratings_movies_df.withColumn('year2', get_year_udf(F.col('title')))\
                 .select('title', 'year', 'year2').show(10, truncate=False)



# Datetimes

Hay varias funciones de _pyspark_ que permiten trabajar con fechas: diferencia entre fechas, dia de la semana, año... Pero para ello primero es necesario transformar las columnas a tipo fecha. Se permite la conversion de dos formatos de fecha:
* timestamp de unix: una columna de tipo entero con los segundos trascurridos entre la medianoche del 1 de Enero de 1990 hasta la fecha.
* cadena: la fecha representada como una cadena siguiendo un formato específico que puede variar.

In [ ]:
ratings_movies_df.select('title', 'timestamp', 'now').show(5)

 

## unix timestamp a datetime

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('datetime', F.from_unixtime(F.col('timestamp')))
ratings_movies_df.select('datetime', 'timestamp').show(10)



## string a datetime

In [ ]:
ratings_movies_df = ratings_movies_df.withColumn('now_datetime', 
                                                 F.from_unixtime(F.unix_timestamp(F.col('now'), 'yyyy/MM/dd HH:mm')))

ratings_movies_df.select('now', 'now_datetime').show(10)



## funciones con datetimes

In [ ]:
ratings_movies_df.select('now_datetime', 'datetime', 
                          F.datediff(F.col('now_datetime'), F.col('datetime'))).show(10)

In [ ]:
ratings_movies_df.select('datetime', F.date_add(F.col('datetime'), 10)).show(10)

In [ ]:
ratings_movies_df.withColumn('datetime_plus_4_months', F.add_months(F.col('datetime'), 4))\
                  .select('datetime', 'datetime_plus_4_months').show(5)

In [ ]:
ratings_movies_df.select('datetime', F.month(F.col('datetime')).alias('month')).show(10)

In [ ]:
ratings_movies_df.select('datetime', F.last_day(F.col('datetime')).alias('last_day')).show(10)

In [ ]:
ratings_movies_df.select('datetime', F.dayofmonth(F.col('datetime')).alias('day'),
                                     F.dayofyear(F.col('datetime')).alias('year_day'),
                                     F.date_format(F.col('datetime'), 'E').alias('weekday')).show(10)



Para filtrar por fechas se pueden comparar directamente con una cadena en el formato YYYY-MM-DD hh:mm:ss ya que será interpretada como una fecha.

In [ ]:
ratings_movies_df.filter(F.col('datetime') >= "2015-09-30 20:00:00").select('datetime', 'title', 'rating').show(10)

In [ ]:
ratings_movies_df.filter(F.col('datetime').between("2003-01-31", "2003-02-10"))\
                  .select('datetime', 'title', 'rating').show(5)

In [ ]:
ratings_movies_df.filter(F.year(F.col('datetime')) >= 2012)\
                 .select('datetime', 'title', 'rating').show(5)



# Ejercicio 1

1) Cree una función que acepte un DataFrame y un diccionario. La función debe usar el diccionario para renombrar un grupo de columnas y devolver el DataFrame ya modificado.

Use el siguiente DataFrame y diccionario:

In [ ]:
pokemon_df = spark.read.csv(DATA_PATH + 'pokemon.csv', sep=',', header=True, inferSchema=True)

rename_dict = {'Sp. Atk': 'sp_atk',
               'Sp. Def': 'sp_def'}

In [ ]:
pokemon_df.show(3)

In [ ]:
# Respuesta

def rename_df(df, rename_dict):
    if any(['.' in c for c in rename_dict.keys()]):
    # withColumnRenamed method
      for old, new in rename_dict.items():
        df = df.withColumnRenamed(old, new)
    else:
      # Select method
      df = df.select([col(c).alias(rename_dict.get(c, c)) for c in df.columns])
    return df



2) Use la función definida en el punto anterior para cambiar los nombres del DF usando el diccionario dado.

3) Modifique la función de tal forma que también acepte una función en lugar de un diccionario. Use la función para renombrar las columnas.

4) Estandarice según las buenas prácticas los nombres de las columnas usando la función que acaba de definir.

5) Cree otra función que acepte un DataFrame y una lista con un subconjunto de columnas. El objetivo de esta función es determinar el número de filas duplicadas del DF.

6) Use la función creada para obtener el número de duplicados del DataFrame pokemon_df en todas las columnas excepto el nombre (`name`)

In [ ]:
# Respuesta

print(pokemon_df.columns)
pokemon_df = rename_df(pokemon_df, rename_dict)
print(pokemon_df.columns)

In [ ]:
# Respuesta

def rename_df(df, rename_object):
    if isinstance(rename_object, dict):
        if any(['.' in c for c in rename_object.keys()]):
           # withColumnRenamed method
           for old, new in rename_object.items():
                df = df.withColumnRenamed(old, new)
        else:
           # Select method
           df = df.select([col(c).alias(rename_object.get(c, c)) for c in df.columns])
    elif isinstance(rename_object, type(lambda x: x)):
        for c in df.columns:
            df = df.withColumnRenamed(c, rename_object(c))
    else:
        raise Exception('Not implemented')
    return df

In [ ]:
# Respuesta
print(pokemon_df.columns)
pokemon_df = rename_df(pokemon_df, lambda c: c.strip().lower().replace('.', '').replace(' ', '_'))
print(pokemon_df.columns)

In [ ]:
# Respuesta
def show_duplicates(df, subset):
    assert isinstance(subset, (list, tuple)), 'Subset is not a list neither a tuple'
  
    agg_count = df.groupBy(subset).count().filter(F.col('count') > 1)
    df = df.join(agg_count, on=subset, how='inner')
    return df

# Note: this could be done more efficiently and with less potential bugs using a window function.

In [ ]:
# Respuesta
subset = [c for c in pokemon_df.columns if c != 'name']

show_duplicates(pokemon_df, subset).show()



# Ejercicio 2

Crea la misma lógica definida en el siguiente UDF, pero sin usar UDFs, es decir, usando exclusivamente funciones de SparkSQL.

In [ ]:
movies_df = spark.read.csv(DATA_PATH + 'movie-ratings/movies.csv', sep=',', header=True, inferSchema=True)
movies_df = movies_df.withColumn('genres', F.split(F.col('genres'), '\|'))

from pyspark.sql.types import StringType, IntegerType, DoubleType, BooleanType

def value_in_col(col, value):
    return value in col

value_in_col_udf = F.udf(value_in_col, BooleanType())



*Pista*: Mira la función *explode*.

In [ ]:
# Respuesta

def filter_by_array_value(df, array_col, value):
    df = df.withColumn('element', F.explode(F.col(array_col)))
    df = df.filter(F.col('element') == value)
    df = df.drop('element')
    return df
  
filter_by_array_value(movies_df, 'genres', 'Drama').show(10, truncate=False)